## Cardiovascular Risk Prediction in Cleveland Population: A Predictive Analysis of Heart Disease Occurrence

## Introduction

Coronary diseases pose significant challenges to everyday wellbeing, requiring precise predictive models where understanding diagnostic factors is crucial. Utilizing a dataset of clinical test results and cardiovascular diagnosis, this analysis focuses on the relationship between age, maximum heart rate, and diagnosis levels in Cleveland. The unique healthcare landscape and lifestyle factors make this exploration essential for tailored interventions and improved cardiovascular risk assessment in the Cleveland population. Thus, this project will explore the following: Can maximum heart rate and age be used to predict heart disease in Clevelanders?

This analysis' heart disease dataset is from a database created to test a discriminant function model, developed using clinical and noninvasive test results from patients undergoing angiography. Collected from groups in Hungary, Switzerland, Long Beach, and Cleveland, the database encompasses variables including demographics, chest pain, resting blood pressure, cholesterol levels, maximum heart rate and other relevant features for diagnosis. The proposed model focuses on the dataset of 303 Cleveland Clinic patients. 

## Methods and Results

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

Warning message:
“package ‘ggplot2’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_co